In [4]:
import os
os.environ['OPENAI_API_KEY'] = ''
os.environ['http_proxy'] = 'http://127.0.0.1:10808'
os.environ['https_proxy'] = 'http://127.0.0.1:10808'

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [3]:
model = ChatOpenAI(model="gpt-4o-mini")
messages = [
    SystemMessage(content="Translate the following from English into Chinese"),
    HumanMessage(content="How are you"),
]

# Invoke only with Language Model

In [8]:
model.input_schema.model_json_schema() # get the pydantic model json of input schema
model.output_schema.model_json_schema() # get the pydantic model json of output schema
print()

In [9]:
model.invoke(messages)

AIMessage(content='你好吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 21, 'total_tokens': 24, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-51c66959-cd79-4184-be96-2043bbf799b7-0', usage_metadata={'input_tokens': 21, 'output_tokens': 3, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

# Invoke with output parsers

In [10]:
parser = StrOutputParser()
result = model.invoke(messages)
parser.invoke(result)

'你好吗？'

In [11]:
chain = model | parser
chain.invoke(messages)

'你好吗？'

# Using Prompt Templates

In [12]:
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)
result = prompt_template.invoke({"language": "Chinese", "text": "hello, this is prompt template"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into Chinese:', additional_kwargs={}, response_metadata={}), HumanMessage(content='hello, this is prompt template', additional_kwargs={}, response_metadata={})])

In [13]:
result.to_messages()

[SystemMessage(content='Translate the following into Chinese:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hello, this is prompt template', additional_kwargs={}, response_metadata={})]

# Chaining prompt templates, models, parsers all

In [14]:
chain = prompt_template | model | parser
chain.invoke({"language": "Chinese", "text": "Chaining all, baby"})

'全部连起来，宝贝。'

In [16]:
chain.input_schema.model_json_schema()

{'properties': {'language': {'title': 'Language', 'type': 'string'},
  'text': {'title': 'Text', 'type': 'string'}},
 'required': ['language', 'text'],
 'title': 'PromptInput',
 'type': 'object'}

In [17]:
chain.output_schema.model_json_schema()

{'title': 'StrOutputParserOutput', 'type': 'string'}

# Serving with langserve, FastAPI, uvicorn


In [15]:
# read:
# https://python.langchain.com/docs/tutorials/llm_chain/